## Homework

The goal of this homework is to get familiar with MLflow, the tool for experiment tracking and model management.

In [1]:
import mlflow
import os

## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

In [ ]:
# In terminal, run:
# pip install -r HW2/HW2_requirements.txt

In [2]:
!mlflow --version

mlflow, version 2.22.0


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* **4**
* 7


In [4]:
! python HW2/preprocess_data.py --raw_data_path HW2/data/ --dest_path HW2/output

In [5]:
!dir "HW2/output" /b

dv.pkl
test.pkl
train.pkl
val.pkl


## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder [`homework`](homework). 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter:

* **2**
* 4
* 8
* 10

In [6]:
! python HW2/train.py --data_path HW2/output


RandomForestRegressor default hyperparameters:
bootstrap: True
ccp_alpha: 0.0
criterion: squared_error
max_depth: 10
max_features: 1.0
max_leaf_nodes: None
max_samples: None
min_impurity_decrease: 0.0
min_samples_leaf: 1
min_samples_split: 2
min_weight_fraction_leaf: 0.0
monotonic_cst: None
n_estimators: 100
n_jobs: None
oob_score: False
random_state: 0
verbose: 0
warm_start: False


## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* **`default-artifact-root`**
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`

In [8]:
#Answer: default-artifact-root

!mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./HW2/artifacts

^C


## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* **5.335**
* 5.818
* 6.336


In [11]:

!python HW2/hpo.py --data_path HW2/output 


100%|██████████| 15/15 [00:50<00:00,  3.40s/trial, best loss: 5.335419588556921]


## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 5.060
* **5.567**
* 6.061
* 6.568


In [15]:
!python HW2/register_model.py --data_path HW2/output 

🏃 View run abundant-cod-320 at: http://127.0.0.1:5000/#/experiments/2/runs/2158eb25441f437ba07aa9b3ab24ae96
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run capable-calf-265 at: http://127.0.0.1:5000/#/experiments/2/runs/cdf6d264127b46048cb672fcf2b16f3a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run polite-ant-248 at: http://127.0.0.1:5000/#/experiments/2/runs/cf1fd2b3e5324b379706138febabeb85
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run delicate-stork-253 at: http://127.0.0.1:5000/#/experiments/2/runs/74d4bd86ef624267a7313e1de2ab4eee
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run clumsy-kite-23 at: http://127.0.0.1:5000/#/experiments/2/runs/4ce553dbe772468a9a53c76f9928a896
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


Successfully registered model 'random-forest-regressor-best'.
2025/05/22 20:44:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random-forest-regressor-best, version 1
Created version '1' of model 'random-forest-regressor-best'.
